In [19]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [20]:
#Read the ods File
path = "../Raw Data/jan20.ods"
df = pd.read_excel(path, engine="odf")
df.head()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,圓⼭⼤飯店,Taipei,500,10667,68.82%,3923,41850353,76393973,131547661
1,TheGrandHotel,Taipei,0,0,NaN,0,0,0,0
2,國賓⼤飯店,Taipei,422,7466,57.07%,3647,27229634,119806163,157208363
3,TheAmbassadorHotel-Taipei,Taipei,0,0,NaN,0,0,0,0
4,台北華國⼤飯店,Taipei,326,6548,64.79%,2936,19225152,28196247,49845151


In [21]:
#Clean Data 
df = pd.DataFrame(df, columns=['Hotel','Area','Room','Demand','Occupancy','ADR','Room_Revenue','F&B_Revenue','Total_Revenue'])
df.head()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,圓⼭⼤飯店,Taipei,500,10667,68.82%,3923,41850353,76393973,131547661
1,TheGrandHotel,Taipei,0,0,NaN,0,0,0,0
2,國賓⼤飯店,Taipei,422,7466,57.07%,3647,27229634,119806163,157208363
3,TheAmbassadorHotel-Taipei,Taipei,0,0,NaN,0,0,0,0
4,台北華國⼤飯店,Taipei,326,6548,64.79%,2936,19225152,28196247,49845151


In [22]:
df = df.dropna()
df

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,圓⼭⼤飯店,Taipei,500,10667,68.82%,3923,41850353,76393973,131547661
2,國賓⼤飯店,Taipei,422,7466,57.07%,3647,27229634,119806163,157208363
4,台北華國⼤飯店,Taipei,326,6548,64.79%,2936,19225152,28196247,49845151
6,華泰王⼦⼤飯店,Taipei,212,3092,47.05%,2753,8511853,17092363,28374158
8,國王⼤飯店,Taipei,97,1434,47.69%,1818,2606300,630428,3286713
...,...,...,...,...,...,...,...,...,...
240,台東桂⽥喜來登酒店,Others,278,4260,49.43%,3517,14984498,15195086,32639251
242,寶華⼤飯店,Others,78,57,2.36%,1242,70800,0,72800
244,⻑榮桂冠酒店(基隆),Others,140,2940,67.74%,3378,9931498,5939549,18889844
246,鈺通⼤飯店,Others,120,1274,34.25%,2177,2774132,9819915,12619573


In [23]:
df.loc[df['Hotel']=='華泰王⼦⼤飯店']

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
6,華泰王⼦⼤飯店,Taipei,212,3092,47.05%,2753,8511853,17092363,28374158


In [24]:
print(df.dtypes)

Hotel            object
Area             object
Room              int64
Demand            int64
Occupancy        object
ADR               int64
Room_Revenue      int64
F&B_Revenue       int64
Total_Revenue     int64
dtype: object


df['Demand'] = df['Demand'].replace(',', '', regex=True)
df['ADR'] = df['ADR'].replace(',', '', regex=True)
df['Room_Revenue'] = df['Room_Revenue'].replace(',', '', regex=True)
df['F&B_Revenue'] = df['F&B_Revenue'].replace(',', '', regex=True)
df['Total_Revenue'] = df['Total_Revenue'].replace(',', '', regex=True)

In [25]:
df['Occupancy'] = df['Occupancy'].str.rstrip('%').astype('float') / 100.0

In [26]:
df[df['Occupancy'].isnull()]

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue


In [27]:
df.astype({'Room': 'int64',
          'Demand': 'int64',
          'Occupancy': 'int64',
          'ADR': 'float64',
          'Room_Revenue': 'int64',
          'F&B_Revenue': 'int64',
          'Total_Revenue': 'int64',
          }).dtypes

Hotel             object
Area              object
Room               int64
Demand             int64
Occupancy          int64
ADR              float64
Room_Revenue       int64
F&B_Revenue        int64
Total_Revenue      int64
dtype: object

In [28]:
Day = 31

In [29]:
#Calculate/Add RevPAR and Demand
df['RevPAR'] = df['Occupancy']*df['ADR']
df['Supply'] = df['Room']*Day
df

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply
0,圓⼭⼤飯店,Taipei,500,10667,0.6882,3923,41850353,76393973,131547661,2699.8086,15500
2,國賓⼤飯店,Taipei,422,7466,0.5707,3647,27229634,119806163,157208363,2081.3429,13082
4,台北華國⼤飯店,Taipei,326,6548,0.6479,2936,19225152,28196247,49845151,1902.2344,10106
6,華泰王⼦⼤飯店,Taipei,212,3092,0.4705,2753,8511853,17092363,28374158,1295.2865,6572
8,國王⼤飯店,Taipei,97,1434,0.4769,1818,2606300,630428,3286713,867.0042,3007
...,...,...,...,...,...,...,...,...,...,...,...
240,台東桂⽥喜來登酒店,Others,278,4260,0.4943,3517,14984498,15195086,32639251,1738.4531,8618
242,寶華⼤飯店,Others,78,57,0.0236,1242,70800,0,72800,29.3112,2418
244,⻑榮桂冠酒店(基隆),Others,140,2940,0.6774,3378,9931498,5939549,18889844,2288.2572,4340
246,鈺通⼤飯店,Others,120,1274,0.3425,2177,2774132,9819915,12619573,745.6225,3720


In [30]:
#Calculate Monthly Room/Supply/Demand/Room Revenue/F&B Revenue/Total Revenue
date = "2020-01-31"
room = round(df["Room"].sum(),0)
supply = round(df["Supply"].sum(),0)
demand = round(df['Demand'].sum(),0)
room_revenue = df["Room_Revenue"].sum()
fb_revenue = df["F&B_Revenue"].sum()
total_revenue = df["Total_Revenue"].sum()

In [31]:
#Calculate Monthly Occupancy/ADR/RevPAR
occ = round((demand/supply),2)
adr = round((room_revenue/demand), 2)
#revpar = round((occ*adr), 2)
revpar = round((room_revenue/supply), 2)

In [32]:
#Append "Total" Row
df = df.append([{'Hotel': 'Total',
                'Area': 'Total',
                'Room': room,
                'Supply': supply,
                'Occupancy': occ,
                'ADR': adr,
                'Room_Revenue': room_revenue,
                'F&B_Revenue': fb_revenue,
                'Total_Revenue': total_revenue,
                'RevPAR': revpar,
                'Demand': demand}], ignore_index=True)
df.tail()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply
121,寶華⼤飯店,Others,78,57,0.0236,1242.00,70800,0,72800,29.3112,2418
122,⻑榮桂冠酒店(基隆),Others,140,2940,0.6774,3378.00,9931498,5939549,18889844,2288.2572,4340
123,鈺通⼤飯店,Others,120,1274,0.3425,2177.00,2774132,9819915,12619573,745.6225,3720
124,陸島酒店,Others,47,114,0.0782,1734.00,197645,14560,212355,135.5988,1457
125,Total,Total,28590,522038,0.5900,4177.49,2180810143,3158882471,5954012828,2460.6100,886290


In [33]:
#Add "Date" Column 
df['Date'] = date
df.tail()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply,Date
121,寶華⼤飯店,Others,78,57,0.0236,1242.00,70800,0,72800,29.3112,2418,2020-01-31
122,⻑榮桂冠酒店(基隆),Others,140,2940,0.6774,3378.00,9931498,5939549,18889844,2288.2572,4340,2020-01-31
123,鈺通⼤飯店,Others,120,1274,0.3425,2177.00,2774132,9819915,12619573,745.6225,3720,2020-01-31
124,陸島酒店,Others,47,114,0.0782,1734.00,197645,14560,212355,135.5988,1457,2020-01-31
125,Total,Total,28590,522038,0.5900,4177.49,2180810143,3158882471,5954012828,2460.6100,886290,2020-01-31


In [34]:
df['Date'] = pd.to_datetime(df['Date']).dt.to_period('D')

In [35]:
#Format 'Summary'
df['Date'] = df['Date'].dt.to_timestamp()
#df = df.style.format({'occ':'{:.0%}',
                    #'demand':'{:.0f}',
                    #'adr':'{:.2f}',
                    #'revpar':'{:.2f}'})

In [36]:
print(df.dtypes)

Hotel                    object
Area                     object
Room                      int64
Demand                    int64
Occupancy               float64
ADR                     float64
Room_Revenue              int64
F&B_Revenue               int64
Total_Revenue             int64
RevPAR                  float64
Supply                    int64
Date             datetime64[ns]
dtype: object


#Create DataFrame 'Summary'
summary = pd.DataFrame({
    'date':date,
    'room':room,
    'supply':supply,
    'demand':demand,
    'room_revenue': room_revenue, 
    'fb_revenue': fb_revenue,
    'total_revenue': total_revenue,
    'occ': occ,
    'adr': adr,
    'revpar':revpar   
}, index=[0])
summary

#Format Summary
summary['date'] = pd.to_datetime(summary['date']).dt.to_period('M')
sep20 = summary.style.format({'occ':'{:.0%}',
                    'demand':'{:.0f}',
                    'adr':'{:.2f}',
                    'revpar':'{:.2f}'})

sep20

df.to_excel(r'static\data\2020\sep20.xlsx', index = False, encoding='utf-8-sig')

In [37]:
df.to_csv(r'..\static\data\2020\jan20.csv', index = False, encoding='utf-8-sig')

In [38]:
from sqlalchemy import create_engine

In [39]:
connection_string = "postgres:postgres@localhost:5432/hotel_db"
engine = create_engine(f'postgresql://{connection_string}', encoding = 'utf-8')

In [40]:
engine.table_names()

['jan20',
 'feb20',
 'mar20',
 'apr20',
 'may20',
 'jun20',
 'jul20',
 'aug20',
 'sep20']

In [41]:
df.to_sql(name='jan20', con=engine, if_exists='append', index=False)